In [1]:
''' RNN 신경망 기본 설정하기 '''

import tensorflow as tf
import numpy as np

# 학습에 필요한 모든 단어를 명시합니다.
char_arr = [c for c in 'SETabcdefghijklmnopqrstuvwxyz고금기동레무발본서선실업의제커툰풍피']
num_dic = {n: i for i, n in enumerate(char_arr)}
dic_len = len(num_dic)

# 입력 단어의 길이와 출력 단어의 길이는 모두 동일합니다.
seq_data = [['actual', '실제'], ['affair', '업무'], ['assent', '동의'],
            ['ballet', '발레'], ['ballon', '풍선'], ['basics', '기본'],
            ['coffee', '커피'], ['coffer', '금고'], ['clumsy', '서툰']]

def make_batch(seq_data):
    input_batch = []
    output_batch = []
    target_batch = []
    
    for seq in seq_data:
        # 입력 값으로 영어 데이터를 지정합니다.
        input = [num_dic[n] for n in seq[0]]
        # 출력 값의 앞에는 'S'를 붙입니다.
        output = [num_dic[n] for n in ('S' + seq[1])]
        # 목표 값의 뒤에는 'E'를 붙입니다.
        target = [num_dic[n] for n in (seq[1] + 'E')]
        
        # 입력 값 및 출력 값은 원-핫 인코딩 방식을 적용합니다.
        input_batch.append(np.eye(dic_len)[input])
        output_batch.append(np.eye(dic_len)[output])
        # 목표 값은 한글 데이터 그 자체의 인덱스로 사용합니다.
        target_batch.append(target)
        
    return input_batch, output_batch, target_batch

In [2]:
''' RNN 학습 모델 구성하기 '''

# RNN 학습 오류 방지를 위해 그래프 리셋 함수를 호출합니다.
tf.reset_default_graph()

# 입력 값 및 출력 값의 형식은 [배치 크기, 스텝, 입력 크기]
enc_input = tf.placeholder(tf.float32, [None, None, dic_len])
dec_input = tf.placeholder(tf.float32, [None, None, dic_len])
# 목표 값의 형식은 [배치 크기, 스텝]입니다.
targets = tf.placeholder(tf.int64, [None, None])

with tf.variable_scope('encode'):
    # 은닉층의 크기를 256로 설정합니다.
    enc_cell = tf.nn.rnn_cell.BasicRNNCell(256)
    enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob=0.7)
    
    outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, enc_input, dtype=tf.float32)

with tf.variable_scope('decode'):
    # 은닉층의 크기를 256로 설정합니다.
    dec_cell = tf.nn.rnn_cell.BasicRNNCell(256)
    dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob=0.7)
    
    # 디코더의 초기 값으로 인코더의 계산 결과인 최종 상태 값을 넣어주어야 합니다.
    outputs, dec_states = tf.nn.dynamic_rnn(dec_cell, dec_input, initial_state=enc_states, dtype=tf.float32)
    
# 출력층을 구성합니다.
model = tf.layers.dense(outputs, dic_len, activation=None)

# 손실 함수 및 최적화 함수를 구성합니다.
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
                      logits=model, labels=targets))
optimizer = tf.train.AdamOptimizer(0.01).minimize(cost)

In [3]:
''' RNN 학습 수행하기 '''

# 학습을 위해 세션 값을 초기화합니다.
sess = tf.Session()
sess.run(tf.global_variables_initializer())

input_batch, output_batch, target_batch = make_batch(seq_data)

# 100회 반복하여 학습을 수행합니다.
for epoch in range(100):
    _, loss = sess.run([optimizer, cost],
                       feed_dict={enc_input: input_batch,
                                  dec_input: output_batch,
                                  targets: target_batch})
    
    print('반복:', '%04d' % (epoch + 1),
          '비용:', '{:.4f}'.format(loss))
    
print('학습 완료!')

반복: 0001 비용: 3.8569
반복: 0002 비용: 1.8925
반복: 0003 비용: 1.1187
반복: 0004 비용: 2.3458
반복: 0005 비용: 0.6461
반복: 0006 비용: 2.1828
반복: 0007 비용: 0.9362
반복: 0008 비용: 0.6453
반복: 0009 비용: 0.8948
반복: 0010 비용: 0.8635
반복: 0011 비용: 0.6073
반복: 0012 비용: 0.5574
반복: 0013 비용: 0.5918
반복: 0014 비용: 0.3767
반복: 0015 비용: 0.4821
반복: 0016 비용: 0.3146
반복: 0017 비용: 0.3367
반복: 0018 비용: 0.2769
반복: 0019 비용: 0.3667
반복: 0020 비용: 0.3243
반복: 0021 비용: 0.2030
반복: 0022 비용: 0.3068
반복: 0023 비용: 0.2302
반복: 0024 비용: 0.3241
반복: 0025 비용: 0.4863
반복: 0026 비용: 0.5911
반복: 0027 비용: 0.3201
반복: 0028 비용: 0.1979
반복: 0029 비용: 0.4117
반복: 0030 비용: 0.3056
반복: 0031 비용: 0.3356
반복: 0032 비용: 0.3500
반복: 0033 비용: 0.2635
반복: 0034 비용: 0.1800
반복: 0035 비용: 0.3227
반복: 0036 비용: 0.1358
반복: 0037 비용: 0.3029
반복: 0038 비용: 0.1641
반복: 0039 비용: 0.2658
반복: 0040 비용: 0.2447
반복: 0041 비용: 0.2293
반복: 0042 비용: 0.3119
반복: 0043 비용: 0.5714
반복: 0044 비용: 0.7572
반복: 0045 비용: 0.4570
반복: 0046 비용: 0.4447
반복: 0047 비용: 0.2629
반복: 0048 비용: 0.8660
반복: 0049 비용: 0.4758
반복: 0050 비용: 0.4175


In [4]:
''' 학습된 모델을 이용해 번역 기능을 사용하기 '''

# 단어를 번역하는 함수를 정의합니다.
def translate(word):
    # 예측 초기에는 한글 단어를 모르므로 한글을 'T'로 채웁니다.
    seq_data = [word, 'T' * len(word)]
    
    input_batch, output_batch, target_batch = make_batch([seq_data])
    
    # 예측 모델을 동작시켜 가장 확률이 높은 글자의 인덱스를 추출합니다.
    prediction = tf.argmax(model, 2)
    
    # 예측 결과는 글자의 인덱스를 뜻하므로 각 글자를 가져와 결과 배열을 만듭니다.
    result = sess.run(prediction,
                      feed_dict={enc_input: input_batch,
                                 dec_input: output_batch,
                                 targets: target_batch})
    decoded = [char_arr[i] for i in result[0]]
    
    # 디코더의 입력 크기만큼 출력값이 나오므로 뒤에 붙은 'E'를 제거합니다.
    end = decoded.index('E')
    translated = ''.join(decoded[:end])
    
    return translated

print('coffee: ', translate('coffee'));
print('actual: ', translate('actual'));
print('affair: ', translate('affair'));
print('clumsy: ', translate('clumsy'));
print('coffer: ', translate('coffer'));
print('ballon: ', translate('ballon'));
print('ballet: ', translate('ballet'));

coffee:  커피
actual:  실제
affair:  업무
clumsy:  서툰
coffer:  금고
ballon:  풍선
ballet:  발레
